# Train a Convolutional Variational Autoencoder 

Using the Conv-VAE package to train a convolutional variational autoencoder on training data collected from the robot. This is put into a notebook, as it needs to be run only once.

The result of running this code is the creation of a "final model". 

The encoder portion of this model is the component that can be used for the encoder of the robot. 

In [1]:
import sys
sys.path.append("..")
from settings import Config
sys.path.append(Config()["conv_vae"]["code_dir"])
# print(Config().values)

import argparse
import numpy as np
import torch
from tqdm import tqdm
import json

# these imports are from the Conv-VAE-Torch package
#import data_loader.data_loaders as module_data
#import model.loss as module_loss
#import model.metric as module_metric
#import model.model as module_arch
#from parse_config import ConfigParser
#from trainer import Trainer
#from utils import prepare_device
# end of Conv-VAE-Torch imports

from torch.nn import functional as F
import torchvision.utils as vutils
from torchvision import transforms
from torch.autograd import Variable
import os
import matplotlib.pyplot as plt
import argparse
import socket

from mpl_toolkits.axes_grid1 import ImageGrid
#Fixes PosixPath Error
import pathlib

# Oh, this hack was fixing something, but for me it is the other way around
#temp = pathlib.PosixPath
#pathlib.PosixPath = pathlib.WindowsPath

Loading pointer config file: /home/lboloni/.config/BerryPicker/mainsettings.yaml
Loading machine-specific config file: /home/lboloni/Insync/lotzi.boloni@gmail.com/Google Drive/LotziStudy/Code/PackageTracking/BerryPicker/settings/settings-tredy2.yaml


In [2]:
from conv_vae import get_config, create_configured_vae_json, train, latest_model, latest_training_run

## Training.

The parameters of the training process are set up in the config file.

This will give an error about the pandas, which is due to certain adressing mode in the Conv-Vae util.py, which probably will stop working in pandas 3.0

In [3]:
file = create_configured_vae_json()
config = get_config(file)
# pretty printing the dict behind the config
print(json.dumps(config.config, indent=4)) 

/home/lboloni/Documents/Hackingwork/_Checkouts/BerryPicker/BerryPicker/src/encoding_conv_vae/conv-vae-config-default.json
{'name': 'VAE_Robot', 'n_gpu': 1, 'arch': {'type': 'VanillaVAE', 'args': {'in_channels': 3, 'latent_dims': 128, 'flow': False}}, 'data_loader': {'type': 'CelebDataLoader', 'args': {'data_dir': '/home/lboloni/Documents/Hackingwork/__Temporary/BerryPicker-training-data/vae-training-data', 'batch_size': 64, 'shuffle': True, 'validation_split': 0.2, 'num_workers': 2}}, 'optimizer': {'type': 'Adam', 'args': {'lr': 0.005, 'weight_decay': 0.0, 'amsgrad': True}}, 'loss': 'elbo_loss', 'metrics': [], 'lr_scheduler': {'type': 'StepLR', 'args': {'step_size': 50, 'gamma': 0.1}}, 'trainer': {'epochs': 3, 'save_dir': '/home/lboloni/Documents/Hackingwork/__Temporary/BerryPicker-models/Conv-VAE/', 'save_period': 1, 'verbosity': 2, 'monitor': 'min val_loss', 'early_stop': 10, 'tensorboard': True}}
{
    "name": "VAE_Robot",
    "n_gpu": 1,
    "arch": {
        "type": "VanillaVAE",


In [4]:
# this is one way to specify the epochs here, but probably not the best
# idea: it should go to the yaml file. 
# 2024-10-16
# I have experienced a mode collapse with the xray training data at 50
config["trainer"]["epochs"] = 100
config["trainer"]["save_period"] = 5

In [5]:
# actually run the training
print(f'Running the trainer from scratch for {config["trainer"]["epochs"]}')
train(config)

Running the trainer from scratch for 100


/home/lboloni/Documents/Hackingwork/_Checkouts/Julian-8897-Conv-VAE-PyTorch/Conv-VAE-PyTorch/utils/util.py:59: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  self._data.total[key] += value * n
/home/lboloni/Documents/Hackingwork/_Checkouts/Ju

In [ ]:
# FIXME: this one returns models, but it should return what was under it.

model_path = pathlib.Path(config["trainer"]["save_dir"], "models", config["name"])

training_run = latest_training_run(model_path)
print(f'The latest training run is {training_run} in directory\n{config["trainer"]["save_dir"]}')

print = latest_model